In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# def initialize_spins(N, theta, noise_amp):
#     
#     spins = np.zeros((N, 3))
#     for j in range(N):
#         # Base azimuthal angle: 0 for even, π for odd
#         base_phi = 0.0 if (j % 2 == 0) else np.pi
#         # Add small noise in [-noise_amp, noise_amp]
#         phi = base_phi + np.random.uniform(-noise_amp, noise_amp)
#         # Spherical coordinates: S = (sinθ cosφ, sinθ sinφ, cosθ)
#         spins[j, 0] = np.sin(theta) * np.cos(phi)  # x component
#         spins[j, 1] = np.sin(theta) * np.sin(phi)  # y component
#         spins[j, 2] = np.cos(theta)                # z component
#     return spins

def initialize_spins(N, theta, noise_amp):
    
    #N = 100            # number of spins in the chain
    delta_ini = 0.1    # amplitude scaling for initial randomness

    theta_random = 2 * np.pi * np.random.random(size=N)
    r_random = delta_ini * np.random.random(size=N)

    Sx = r_random * np.cos(theta_random)
    Sy = r_random * np.sin(theta_random)

    
    Sz_magnitude = np.sqrt(1. - Sx**2 - Sy**2)
    alternating_sign = (-1) ** np.arange(N)
    Sz = Sz_magnitude * alternating_sign
    spins = np.column_stack((Sx, Sy, Sz))
    return spins



In [ ]:
def compute_H_ave(spins, J, h, g):
    """
    Compute the time-averaged Hamiltonian:
      H_ave = 1/2 * sum_j [ J S_{j,x} S_{j+1,x} + h S_{j,z} + g S_{j,x} ]
    with periodic boundary conditions.
    """
    Sx = spins[:, 0]
    Sz = spins[:, 2]
    bond_term = J * (Sz * np.roll(Sz, -1))
    field_term_z = h * Sz
    field_term_x = g * Sx
    H = 0.5 * np.sum(bond_term + field_term_z + field_term_x)
    return H



In [ ]:

def rotate_z(spins, phi):
    x_old = spins[:,0].copy()
    y_old = spins[:,1].copy()
    spins[:,0] = x_old*np.cos(phi) - y_old*np.sin(phi)
    spins[:,1] = x_old*np.sin(phi) + y_old*np.cos(phi)
    return spins

def rotate_x(spins, theta):
    x_old = spins[:,0].copy()
    y_old = spins[:,1].copy()
    z_old = spins[:,2].copy()
    spins[:,1] = y_old*np.cos(theta) - z_old*np.sin(theta)
    spins[:,2] = y_old*np.sin(theta) + z_old*np.cos(theta)
    return spins



In [ ]:
def update_spins(spins, J, h, g, T):

    Sz = spins[:, 2]
    kappa = J * (np.roll(Sz, 1) + np.roll(Sz, -1)) + h
    angle1 = (kappa * T)/2
    spins1 = rotate_z(spins,angle1)
    spins2 = rotate_x(spins1, (g * T)/2)
    return spins2


In [ ]:
# def update_spins(spins, J, h, g, T):
#     

#     dt = T / 2  # half-period
    
#     # ---- First half: Rotation about z ----
#     Sx = spins[:, 0]
#     Sz = spins[:, 2]
#     # Compute spin-dependent frequency κ_j using periodic BC:
#     kappa = J * (np.roll(Sz, 1) + np.roll(Sz, -1)) + h
#     angle1 = kappa * dt
#     cos1 = np.cos(angle1)
#     sin1 = np.sin(angle1)
    
#     # Apply rotation in the (x, y) plane:
#     x_old = spins[:, 0].copy()
#     y_old = spins[:, 1].copy()
#     spins[:, 0] = x_old * cos1 - y_old * sin1
#     spins[:, 1] = x_old * sin1 + y_old * cos1
#     # spins[:,2] remains unchanged
    
#     # ---- Second half: Rotation about x ----
#     angle2 = g * dt
#     cos2 = np.cos(angle2)
#     sin2 = np.sin(angle2)
    
#     # Apply rotation in the (y, z) plane:
#     y_old = spins[:, 1].copy()
#     z_old = spins[:, 2].copy()
#     spins[:, 1] = y_old * cos2 - y_old * sin2
#     spins[:, 2] = y_old * sin2 + z_old * cos2
#     # spins[:,0] remains unchanged
    
#     return spins



In [ ]:
def simulate_run(L, N, J, h, g, T, theta, noise_amp):
    """
    Simulate one run for L driving cycles.
    Returns an array Q_vals with Q(lT) for l=0,...,L.
    """
    spins = initialize_spins(N, theta, noise_amp)
    Q_vals = np.zeros(L + 1)
    Q_vals[0] = 0.0  # At t=0, Q = 0 (by definition)
    
    # Evolve for L cycles.
    for l in range(1, L + 1):
        spins = update_spins(spins, J, h, g, T)
        H = compute_H_ave(spins, J, h, g)
        Q_vals[l] = (H)
        #print(f"Cycle {l}: Q = {Q_vals[l]}")
    return Q_vals



In [80]:
def ensemble_average(num_runs, L, N, J, h, g, T, theta, noise_amp,E0):
    """
    Average Q(lT) over an ensemble of num_runs independent realizations.
    """
    Q_ensemble = np.zeros(L + 1)
    for run in range(num_runs):
        Q_run= simulate_run(L, N, J, h, g, T, theta, noise_amp)
        Q_ensemble += Q_run
    Q_ensemble /= num_runs
    Q_ensemble = (Q_ensemble - E0)/(-E0)
    return Q_ensemble



In [ ]:
def main():
    # -- Simulation parameters --
    N = 100            # number of spins
    L = 2000000      # number of driving cycles
    J = 1.0
    g = 0.809
    h = 0.7045
    Omega = 4.6       # driving frequency (in units of J)
    T = 2 * np.pi / Omega  # period T = 2π/Omega
    theta = np.pi / 4  # choose polar angle π/2 so that spins lie mostly in the xy-plane;
    noise_amp = np.pi / 100  # small noise in the azimuthal angle
    num_runs = 50      # ensemble size
    spins = initialize_spins(N, theta, noise_amp)
    E0 = compute_H_ave(spins, J, h, g)
    # Compute ensemble-averaged Q(lT)
    Q_avg = ensemble_average(num_runs, L, N, J, h, g, T, theta, noise_amp,E0)

    cycles = np.arange(L + 1)
    
    # Plot Q vs. number of cycles.
    plt.figure(figsize=(8, 6))
    plt.plot(cycles, Q_avg, label=r'$\langle Q(\ell T)\rangle$')
    plt.xlabel('Driving cycles, ℓ')
    plt.ylabel(r'$Q(\ell T)$')
    plt.title('Energy absorption in the driven spin chain')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    main()